# Importing Libraries

In [18]:
import boto3
import os
import time
from trp import Document
from botocore.config import Config
from PIL import Image, ImageDraw

import warnings
warnings.filterwarnings("ignore")


import calendar
import re
import nltk
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from transformers import AutoTokenizer,GPT2TokenizerFast

import os
import openai
openai.api_key =os.getenv('OPENAI_API_KEY')

# Extract text from a image

### Image Text Extractor

In [6]:
def image_text_extractor(path:str)->str:
    '''
    This function extracts text from a given image 
    '''
    with open(path, 'rb') as image:
        img = bytearray(image.read())
        client = boto3.client('textract',region_name='ap-south-1')
        response = client.detect_document_text(
            Document={'Bytes': img}
        )
    text = ""
    for item in response["Blocks"]:
        if item["BlockType"] == "LINE":
            text = text + " "+item["Text"]

    return text

### Function test

In [7]:
text = image_text_extractor('/home/cloudcraftz/Desktop/Test_3.jpg')

In [8]:
text

" Multi-Objective reward generalization: Improving performance of Deep Reinforcement Learning for selected applications in stock and cryptocurrency trading Federico Cornalba* Constantin Disselkamp**S Institute of Science and Technology Austria Davide Scassola** federico.cornalba@ist.ac.at Christopher Helf TRALITY GmbH constantin@trality.com davide@trality.com christopher@trality.com ABSTRACT the state variables and action (s,a determine the next state of the We investigate the potential of Multi-Objective, Deep Reinforce- environment that the algorithm will visit. The algorithm's ultimate ment Learning for stock and cryptocurrency trading. More specifi- goal is to maximise a cumulative reward (accounting for all actions cally, we build on the generalized setting à la Fontaine and Friedman taken in a given episode), which is in turn based on a pre-specified [6] (where the reward weighting mechanism is not specified a pri- state-action reward function r (assigning a numerical reward r(s,

# Extract text from a document

### Document Text Extractor

In [10]:
def document_text_extractor(name:str)->list:
    ''' 
    This function takes the name of the document stored in AWS S3 and returns the actual text, the extracted text in a list.
    '''

    def InvokeTextDetectJob(s3BucketName:str, objectName:str):
        response = None
        client = boto3.client('textract',region_name='ap-south-1')
        
        
        response = client.start_document_text_detection(
                DocumentLocation={
                        'S3Object': {
                                        'Bucket': s3BucketName,
                                        'Name': objectName,
                                    }
            })
        return response["JobId"]


    def CheckJobComplete(jobId):
        time.sleep(5)
        client = boto3.client('textract',region_name='ap-south-1')
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
        while(status == "IN_PROGRESS"):
            time.sleep(5)
            response = client.get_document_text_detection(JobId=jobId)
            status = response["JobStatus"]
            print("Job status: {}".format(status))
        return status


    def JobResults(jobId):
        pages = []
        client = boto3.client('textract',region_name='ap-south-1')
        response = client.get_document_text_detection(JobId=jobId)
    
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']
            while(nextToken):
                response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
                pages.append(response)
                print("Resultset page recieved: {}".format(len(pages)))
                nextToken = None
                if('NextToken' in response):
                    nextToken = response['NextToken']
        return pages    


    extracted_text = []

    # S3 Document Data
    s3BucketName = "fintech-sentiment-textract"
    documentName = name

    # Function invokation
    jobId = InvokeTextDetectJob(s3BucketName, documentName)
    print(f"Started job with id: {jobId} , name : {name}")

    if(CheckJobComplete(jobId)):
        response = JobResults(jobId)
        text_out = ''
        for resultPage in response:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    text_out = text_out + item["Text"]
                    text_out = text_out.replace("\'s","")

    print(f"For {name}, Text Extracted")
    extracted_text.append(text_out)

    return extracted_text


### Function test

In [11]:
extracted_text = document_text_extractor('RL-1.pdf')

Started job with id: f90ec13d3a0bfe70d5fd7f91158d77156169096aea1d7985323a1dd5a5b20d92 , name : RL-1.pdf
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
For RL-1.pdf, Text Extracted


In [12]:
extracted_text

['Multi-Objective reward generalization: Improving performanceof Deep Reinforcement Learning for selected applications instock and cryptocurrency tradingFederico Cornalba*Constantin Disselkamp**SInstitute of Science and Technology AustriaDavide Scassola**federico.cornalba@ist.ac.atChristopher HelfTRALITY GmbHconstantin@trality.comdavide@trality.comchristopher@trality.comABSTRACTthe state variables and action (s, determine the next state of theWe investigate the potential of Multi-Objective, Deep Reinforce-environment that the algorithm will visit. The algorithm ultimatement Learning for stock and cryptocurrency trading. More specifi-goal is to maximise a cumulative reward (accounting for all actionscally, we build on the generalized setting à la Fontaine and Friedmantaken in a given episode), which is in turn based on a pre-specified[6] (where the reward weighting mechanism is not specified a pri-state-action reward function r (assigning a numerical reward r(s, a)ori, but embedded in t

# Chat

### Question Answer

In [19]:
def trained_answer(question:str)->str:
    ''' 
    This function provides answer to a question from the data it has been trained on.
    '''

    COMPLETIONS_MODEL = "text-davinci-003"
    answer=openai.Completion.create(
        prompt=question,
        temperature=0,
        max_tokens=300,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        model=COMPLETIONS_MODEL
    )["choices"][0]["text"].strip(" \n")
    ModelAnswer=answer

    return ModelAnswer    


def answer_the_question(text:str,question:str)->str:  
    ''' 
    This function answers a question based on the given context.
    '''  
    
    COMPLETIONS_MODEL = "text-davinci-003"
    context=text
    prompt = f"""Answer the question as truthfully as possible with structured compact english sentences using the provided text, and if the answer is not contained within the text below, say "not_found"
        Context:
        '{context}'
        Q: "{question}"?
        A:"""
    answer= openai.Completion.create(
        prompt=prompt,
        temperature=0,
        max_tokens=300,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        model=COMPLETIONS_MODEL
        )["choices"][0]["text"].strip(" \n")
    
    if answer=='Not_found.' or answer == 'not_found':
        return trained_answer(question)
    else:
        return answer
        


### Function test

In [17]:
answer_the_question(text,'What is sharpe ratio?')

"The Sharpe ratio is a measure of risk-adjusted return, which compares the return of an investment to that of an alternative investment with similar risk. It is calculated by subtracting the risk-free rate from the rate of return of the investment and dividing the result by the standard deviation of the investment's returns. The higher the Sharpe ratio, the better the investment's performance has been relative to the amount of risk taken."

In [21]:
answer_the_question(text,'Give me a summary of this paper?')

"This paper investigates the potential of Multi-Objective, Deep Reinforcement Learning for stock and cryptocurrency trading. It builds on the generalized setting à la Fontaine and Friedman by complementing it with computational speed-ups, and adding the cumulative reward's discount factor to the learning process. The paper verifies that the resulting Multi-Objective algorithm generalizes well and has a clear edge over the corresponding Single-Objective strategy when the reward mechanism is sparse. The code is provided in open source format."

In [22]:
answer_the_question(text,'What is cryptocurrency trading ?')

'Cryptocurrency trading is the buying and selling of digital currencies, such as Bitcoin, Ethereum, and Litecoin, in order to generate a profit.'